In [1]:
import astropy.units as u
import astropy.constants as C
import importlib
import units.smm as smu
import logging
importlib.reload(smu)

<module 'units.smm' from 'e:\\Code\\autism\\ss13\\supermattercore\\units\\smm.py'>

In [2]:
smu.PumpLogger.setLevel(logging.DEBUG)
%config Application.logging_config = {'loggers': {'ct.sm': {'propagate': True, 'level': 'DEBUG', 'handlers': ['console']}}}

In [3]:
core_room = smu.GasMixture(smu.T_MINIMAL, 15*smu.CELL_VOLUME, {})
smcore = smu.Supermatter(core_room, smu.POWERED_EMITTER_DMG*8 / smu.SM_EER_EMIT)

In [4]:
canist_ref = smu.GasMixture.pure('N2', 293.15*u.K, 1000*u.L, 90*smu.atm)
canist_ref

GasMixture(T=<Quantity 293.15 K>, V=<Quantity 1000. l>, nus={'N2': <Quantity 3743.41692471 mol>})

In [5]:
cooled = smu.GasMixture(smu.T_MINIMAL, 70*u.L)
cooling = smu.GasMixture(smu.T_MINIMAL, 15680*u.L)
warm = smu.GasMixture(smu.T_MINIMAL, 1890*u.L)
scalding = smu.GasMixture(smu.T_MINIMAL, 315*u.L)

In [6]:
po1 = smu.Pump('PO1', P_max=45*u.kW, V_out=0*u.L, V_in=0.7*u.m**3) # Outlet Injector
pi1 = smu.Pump('PI1', V_in=0*u.L, V_out=0.2*u.m**3) # Vent Pump
hp1 = smu.Pump('HP1', P_max=45*u.kW)
pc_hot =  smu.Pump('P[C]hot') # C2-P2 on pic
pc_cold = smu.Pump('P[C]coldd') # C5-P6 on pic
thot =  smu.Turbine('T(hot)1')
tcold = smu.Turbine('T(cold)2')
teg = smu.TEG(thot, tcold)
he = smu.HeatExchanger(191)

In [7]:
pc_hot.connect(canist_ref*1, warm)
pc_cold.connect(canist_ref*1, cooling)
hp1.connect(cooling, cooled)
pi1.connect(core_room, scalding)
po1.connect(warm, core_room)
thot.connect(scalding, warm)
tcold.connect(cooled, cooling)
warm.V += 2*u.m**3 # Filters

In [8]:
%config Application.log_level = 'INFO'
for _ in range(5):
    print([x.pump() for x in [pc_hot, pc_cold, po1, pi1]])
    print(teg.process())
    (he.cool(cooling))
%config Application.log_level = 30
print(pc_hot.inlet.p.to(u.kPa), pc_cold.inlet.p.to(u.kPa))
print(cooling.p.to(u.kPa), warm.p.to(u.kPa), scalding.p.to(u.kPa))

[ct.sm.Pump] P[C]hot: Source entropy 155.129 J / (K mol) -> Sink entropy 150000 J / (K mol)
[ct.sm.Pump] P[C]hot: Specific entropy change 149845 J / (K mol)
[ct.sm.Pump] P[C]hot: Specific power 0 J / mol
[ct.sm.Pump] P[C]hot: Transferred 623.903 mol in fact
[ct.sm.Pump] P[C]coldd: Source entropy 155.129 J / (K mol) -> Sink entropy 150000 J / (K mol)
[ct.sm.Pump] P[C]coldd: Specific entropy change 149845 J / (K mol)
[ct.sm.Pump] P[C]coldd: Specific power 0 J / mol
[ct.sm.Pump] P[C]coldd: Transferred 623.903 mol in fact
[ct.sm.Pump] PO1: Source entropy 181.655 J / (K mol) -> Sink entropy 150000 J / (K mol)
[ct.sm.Pump] PO1: Specific entropy change 149818 J / (K mol)
[ct.sm.Pump] PO1: Specific power 0 J / mol
[ct.sm.Pump] PO1: Transferred 87.5214 mol in fact
[ct.sm.Pump] PI1: Source entropy 214.728 J / (K mol) -> Sink entropy 150000 J / (K mol)
[ct.sm.Pump] PI1: Specific entropy change 149785 J / (K mol)
[ct.sm.Pump] PI1: Specific power 0 J / mol
[ct.sm.Pump] PI1: Transferred 87.5214 mol 

[<Quantity 0. J>, <Quantity 0. J>, <Quantity 0. J>, <Quantity 0. J>]
0.0 J


[ct.sm.Pump] P[C]hot: Source entropy 156.608 J / (K mol) -> Sink entropy 182.909 J / (K mol)
[ct.sm.Pump] P[C]hot: Specific entropy change 26.3012 J / (K mol)
[ct.sm.Pump] P[C]hot: Specific power 0 J / mol
[ct.sm.Pump] P[C]hot: Transferred 519.919 mol in fact
[ct.sm.Pump] P[C]coldd: Source entropy 156.608 J / (K mol) -> Sink entropy 191.476 J / (K mol)
[ct.sm.Pump] P[C]coldd: Specific entropy change 34.8682 J / (K mol)
[ct.sm.Pump] P[C]coldd: Specific power 0 J / mol
[ct.sm.Pump] P[C]coldd: Transferred 519.919 mol in fact
e:\Code\autism\ss13\.venv\lib\site-packages\astropy\units\quantity.py:671: RuntimeWarning: invalid value encountered in divide
  result = super().__array_ufunc__(function, method, *arrays, **kwargs)
[ct.sm.Pump] PO1: Source entropy 177.285 J / (K mol) -> Sink entropy nan J / (K mol)
[ct.sm.Pump] PO1: Specific entropy change nan J / (K mol)
[ct.sm.Pump] PO1: Specific power 0 J / mol
[ct.sm.Pump] PO1: Transferred 148.178 mol in fact
[ct.sm.Pump] PI1: Source entropy 210.

[<Quantity 0. J>, <Quantity 0. J>, <Quantity 0. J>, <Quantity 30000. J>]
0.0 J


[ct.sm.Pump] P[C]hot: Source entropy 158.094 J / (K mol) -> Sink entropy 178.539 J / (K mol)
[ct.sm.Pump] P[C]hot: Specific entropy change 20.4454 J / (K mol)
[ct.sm.Pump] P[C]hot: Specific power 0 J / mol
[ct.sm.Pump] P[C]hot: Transferred 433.266 mol in fact
[ct.sm.Pump] P[C]coldd: Source entropy 158.094 J / (K mol) -> Sink entropy 186.442 J / (K mol)
[ct.sm.Pump] P[C]coldd: Specific entropy change 28.3486 J / (K mol)
[ct.sm.Pump] P[C]coldd: Specific power 0 J / mol
[ct.sm.Pump] P[C]coldd: Transferred 433.266 mol in fact
[ct.sm.Pump] PO1: Source entropy 175.304 J / (K mol) -> Sink entropy 210.913 J / (K mol)
[ct.sm.Pump] PO1: Specific entropy change 35.6097 J / (K mol)
[ct.sm.Pump] PO1: Specific power 0 J / mol
[ct.sm.Pump] PO1: Transferred 188.171 mol in fact
[ct.sm.Pump] PI1: Source entropy 203.783 J / (K mol) -> Sink entropy 183.009 J / (K mol)
[ct.sm.Pump] PI1: Specific entropy change -20.7748 J / (K mol)
[ct.sm.Pump] PI1: Specific power 2436.05 J / mol
[ct.sm.Pump] PI1: Transferr

[<Quantity 0. J>, <Quantity 0. J>, <Quantity 0. J>, <Quantity 30000. J>]
0.0 J


[ct.sm.Pump] P[C]hot: Source entropy 159.584 J / (K mol) -> Sink entropy 176.557 J / (K mol)
[ct.sm.Pump] P[C]hot: Specific entropy change 16.973 J / (K mol)
[ct.sm.Pump] P[C]hot: Specific power 0 J / mol
[ct.sm.Pump] P[C]hot: Transferred 361.055 mol in fact
[ct.sm.Pump] P[C]coldd: Source entropy 159.584 J / (K mol) -> Sink entropy 183.776 J / (K mol)
[ct.sm.Pump] P[C]coldd: Specific entropy change 24.1917 J / (K mol)
[ct.sm.Pump] P[C]coldd: Specific power 0 J / mol
[ct.sm.Pump] P[C]coldd: Transferred 361.055 mol in fact
[ct.sm.Pump] PO1: Source entropy 174.299 J / (K mol) -> Sink entropy 204.103 J / (K mol)
[ct.sm.Pump] PO1: Specific entropy change 29.8042 J / (K mol)
[ct.sm.Pump] PO1: Specific power 0 J / mol
[ct.sm.Pump] PO1: Transferred 212.423 mol in fact
[ct.sm.Pump] PI1: Source entropy 199.813 J / (K mol) -> Sink entropy 182.018 J / (K mol)
[ct.sm.Pump] PI1: Specific entropy change -17.7951 J / (K mol)
[ct.sm.Pump] PI1: Specific power 2086.65 J / mol
[ct.sm.Pump] PI1: Transferre

[<Quantity 0. J>, <Quantity 0. J>, <Quantity 0. J>, <Quantity 30000. J>]
0.0 J


[ct.sm.Pump] P[C]hot: Specific entropy change 14.4733 J / (K mol)
[ct.sm.Pump] P[C]hot: Specific power 0 J / mol
[ct.sm.Pump] P[C]hot: Transferred 300.879 mol in fact
[ct.sm.Pump] P[C]coldd: Source entropy 161.078 J / (K mol) -> Sink entropy 182.065 J / (K mol)
[ct.sm.Pump] P[C]coldd: Specific entropy change 20.9865 J / (K mol)
[ct.sm.Pump] P[C]coldd: Specific power 0 J / mol
[ct.sm.Pump] P[C]coldd: Transferred 300.879 mol in fact
[ct.sm.Pump] PO1: Source entropy 173.828 J / (K mol) -> Sink entropy 200.043 J / (K mol)
[ct.sm.Pump] PO1: Specific entropy change 26.2151 J / (K mol)
[ct.sm.Pump] PO1: Specific power 0 J / mol
[ct.sm.Pump] PO1: Transferred 224.832 mol in fact
[ct.sm.Pump] PI1: Source entropy 197.021 J / (K mol) -> Sink entropy 180.994 J / (K mol)
[ct.sm.Pump] PI1: Specific entropy change -16.0264 J / (K mol)
[ct.sm.Pump] PI1: Specific power 1879.26 J / mol
[ct.sm.Pump] PI1: Transferred 15.9638 mol in fact
[ct.sm] T(cold)2: merged: self.inlet.p=<Quantity 0. Pa>, self.outlet.p

[<Quantity 0. J>, <Quantity 0. J>, <Quantity 0. J>, <Quantity 30000. J>]
0.0 J
3054.0183738425935 kPa 3054.0183738425935 kPa
334.52292608031905 kPa 672.6776532537343 kPa 372.30975368073933 kPa


In [128]:
%config Application.log_level = 'INFO'
for _ in range(5):
    pump.pump(canist, loop)
%config Application.log_level = 30

NameError: name 'pump' is not defined

[IPKernelApp] Exception in execute request:
---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
Cell In[128], line 3
      1 get_ipython().run_line_magic('config', "Application.log_level = 'INFO'")
      2 for _ in range(5):
----> 3     pump.pump(canist, loop)
      4 get_ipython().run_line_magic('config', 'Application.log_level = 30')

NameError: name 'pump' is not defined
